In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
# create the session
conf = SparkConf().set("spark.ui.port", "4050")
import pyspark
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
openjdk-8-jdk-headless is already the newest version (8u352-ga-1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.
--2023-01-19 10:12:13--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 54.161.241.46, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  17.4MB/s    in 0.8s    

2023-01-19 10:12:14 (17.4 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://7601-35-227-142-233.ngrok.io


Q1

In [2]:
df = spark.read.json("sample_books.json")
df.printSchema()

root
 |-- author: string (nullable = true)
 |-- edition: string (nullable = true)
 |-- price: double (nullable = true)
 |-- title: string (nullable = true)
 |-- year_written: long (nullable = true)



In [3]:
df.show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Twain, Mark|          Penguin| 5.76|    Huckleberry Finn|        1865|
|    Dickens, Charles|     Random House| 5.75|         Bleak House|        1870|
|         Twain, Mark|     Random House| 7.75|          Tom Sawyer|        1862|
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Rowling, J.K.|   Har

In [4]:
df.count()

13

In [5]:
df.select('title', 'price').show()

+--------------------+-----+
|               title|price|
+--------------------+-----+
|    Northanger Abbey| 18.2|
|       War and Peace| 12.7|
|       Anna Karenina| 13.5|
|       Mrs. Dalloway| 25.0|
|           The Hours|12.35|
|    Huckleberry Finn| 5.76|
|         Bleak House| 5.75|
|          Tom Sawyer| 7.75|
| A Room of One's Own| 29.0|
|        Harry Potter|19.95|
|One Hundred Years...| 14.0|
|Hamlet, Prince of...| 7.95|
|   Lord of the Rings|27.45|
+--------------------+-----+



In [6]:
df.filter((df.year_written>1950) & (df.price>10)).show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|       Rowling, J.K.|   Harcourt Brace|19.95|        Harry Potter|        2000|
|             Marquez|Harper  Perennial| 14.0|One Hundred Years...|        1967|
+--------------------+-----------------+-----+--------------------+------------+



In [7]:
df2=df.filter((df.year_written>1950) & (df.price>10))
df2.select('title', 'price', 'year_written').show()

+--------------------+-----+------------+
|               title|price|year_written|
+--------------------+-----+------------+
|           The Hours|12.35|        1999|
|        Harry Potter|19.95|        2000|
|One Hundred Years...| 14.0|        1967|
+--------------------+-----+------------+



In [8]:
df.filter(df.title=='Harry Potter').select('title','year_written').show()

+------------+------------+
|       title|year_written|
+------------+------------+
|Harry Potter|        2000|
+------------+------------+



In [9]:
df.orderBy(['price'], ascending = [False]).show()

+--------------------+-----------------+-----+--------------------+------------+
|              author|          edition|price|               title|year_written|
+--------------------+-----------------+-----+--------------------+------------+
|     Woolf, Virginia|          Penguin| 29.0| A Room of One's Own|        1922|
|       Tolkien, J.R.|          Penguin|27.45|   Lord of the Rings|        1937|
|     Woolf, Virginia|   Harcourt Brace| 25.0|       Mrs. Dalloway|        1925|
|       Rowling, J.K.|   Harcourt Brace|19.95|        Harry Potter|        2000|
|        Austen, Jane|          Penguin| 18.2|    Northanger Abbey|        1814|
|             Marquez|Harper  Perennial| 14.0|One Hundred Years...|        1967|
|        Tolstoy, Leo|          Penguin| 13.5|       Anna Karenina|        1875|
|        Tolstoy, Leo|          Penguin| 12.7|       War and Peace|        1865|
|Cunnningham, Michael|   Harcourt Brace|12.35|           The Hours|        1999|
|         Shakespeare| Signe

Q2

In [10]:
mtcars=spark.read.csv('/content/mtcars.csv', header=True)
mtcars.show(10)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|            model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|  21|  6|  160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|  21|  6|  160|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|  108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|  258|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|  360|175|3.15| 3.44|17.02|  0|  0|   3|   2|
|          Valiant|18.1|  6|  225|105|2.76| 3.46|20.22|  1|  0|   3|   1|
|       Duster 360|14.3|  8|  360|245|3.21| 3.57|15.84|  0|  0|   3|   4|
|        Merc 240D|24.4|  4|146.7| 62|3.69| 3.19|   20|  1|  0|   4|   2|
|         Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|
|         Merc 280|19.2|  6|167.6|123|3.92| 3.44| 18.3|  1|  0|   4|   4|
+-----------------+----+---+-----+---+

In [11]:
mtcars.select('mpg').show(5)

+----+
| mpg|
+----+
|  21|
|  21|
|22.8|
|21.4|
|18.7|
+----+
only showing top 5 rows



In [12]:
mtcars.filter(mtcars.mpg < 18).show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|         Duster 360|14.3|  8|  360|245|3.21| 3.57|15.84|  0|  0|   3|   4|
|          Merc 280C|17.8|  6|167.6|123|3.92| 3.44| 18.9|  1|  0|   4|   4|
|         Merc 450SE|16.4|  8|275.8|180|3.07| 4.07| 17.4|  0|  0|   3|   3|
|         Merc 450SL|17.3|  8|275.8|180|3.07| 3.73| 17.6|  0|  0|   3|   3|
|        Merc 450SLC|15.2|  8|275.8|180|3.07| 3.78|   18|  0|  0|   3|   3|
| Cadillac Fleetwood|10.4|  8|  472|205|2.93| 5.25|17.98|  0|  0|   3|   4|
|Lincoln Continental|10.4|  8|  460|215|   3|5.424|17.82|  0|  0|   3|   4|
|  Chrysler Imperial|14.7|  8|  440|230|3.23|5.345|17.42|  0|  0|   3|   4|
|   Dodge Challenger|15.5|  8|  318|150|2.76| 3.52|16.87|  0|  0|   3|   2|
|        AMC Javelin|15.2|  8|  304|150|3.15|3.435| 17.3|  0|  0|   3|   2|
|         Ca

In [13]:
mtcars.withColumn('wtTon', mtcars['wt'] * 0.45).show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+------------------+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|             wtTon|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+------------------+
|          Mazda RX4|  21|  6|  160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|             1.179|
|      Mazda RX4 Wag|  21|  6|  160|110| 3.9|2.875|17.02|  0|  1|   4|   4|           1.29375|
|         Datsun 710|22.8|  4|  108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|             1.044|
|     Hornet 4 Drive|21.4|  6|  258|110|3.08|3.215|19.44|  1|  0|   3|   1|           1.44675|
|  Hornet Sportabout|18.7|  8|  360|175|3.15| 3.44|17.02|  0|  0|   3|   2|             1.548|
|            Valiant|18.1|  6|  225|105|2.76| 3.46|20.22|  1|  0|   3|   1|             1.557|
|         Duster 360|14.3|  8|  360|245|3.21| 3.57|15.84|  0|  0|   3|   4|            1.6065|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19

In [14]:
mtcars.groupby(['cyl']).agg({"wt": "AVG"}).show()

+---+------------------+
|cyl|           avg(wt)|
+---+------------------+
|  8|3.9992142857142867|
|  6| 3.117142857142857|
|  4| 2.285727272727273|
+---+------------------+



In [15]:
df2=mtcars.groupby(['cyl']).agg({"wt": "AVG"})
df2.orderBy(['cyl'], ascending = [True]).show()

+---+------------------+
|cyl|           avg(wt)|
+---+------------------+
|  4| 2.285727272727273|
|  6| 3.117142857142857|
|  8|3.9992142857142867|
+---+------------------+



In [16]:
mtcars.createTempView("cars")
df2=spark.sql("select gear from cars where cyl between 4 and 9")
df2.show()

+----+
|gear|
+----+
|   4|
|   4|
|   4|
|   3|
|   3|
|   3|
|   3|
|   4|
|   4|
|   4|
|   4|
|   3|
|   3|
|   3|
|   3|
|   3|
|   3|
|   4|
|   4|
|   4|
+----+
only showing top 20 rows

